In [6]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
sc = spark.sparkContext

## Spark SQL

* En Spark 1.6 se introdujo una nueva abstracción de programación llamada API estructurada. Esta es la forma preferida para realizar el procesamiento de datos en la mayoría de los casos de uso.

* En esta nueva forma de hacer el procesamiento de datos, los datos deben organizarse en un formato estructurado y la lógica de cálculo de datos debe seguir una determinada estructura. Con estas dos piezas de información, Spark puede realizar optimizaciones para acelerar las aplicaciones de procesamiento de datos.

* El componente SparkSQL está construido sobre el viejo y confiable componente SparkCore. Esta arquitectura en capas significa que cualquier mejora en el componente Spark Core estará disponible automáticamente para el componente para SQL.

* El concepto DF se inspiró en el concepto de pandasDF de python. La principal diferencia es que un DF en Spark puede manejar un gran volumen de datos que se distribuyen en muchas máquinas.

* Un concepto fundamental que diferencia a los datos estructurados de los no estructurados es un **esquema** que define la estructura de los datos en forma de nombres de columna y tipos de datos asociados. El concepto de **esquema** es una parte integral de las APIs estructuradas de Spark.

* Los datos estructurados a menudo se capturan en un formato determinado. Algunos de estos formatos están basados en textos y algunos de ellos están basados en binario.

    * Los formatos comunes para datos de texto son CSV, XML y JSON 
    * Los formatos comunes para datos binarios son agro, parquet y ORC.
<br>.
* El módulo SparkSQL y facilita la lectura y escritura de datos desde y hacia cualquiera de estos formatos. Un beneficio inesperado que surge de esta versatilidad es que Spark se puede utilizar como una herramienta de conversión de formato de datos.

### Crear un DF a partir de un RDD

Hay muchas formas de crear un DF, pero **siempre hay que proporcionar un esquema**, ya sea implícita o explícitamente.

In [3]:
rdd = sc.parallelize([item for item in range(10)]).map(lambda x: (x, x ** 2))

rdd.collect()

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]

In [5]:
# Para crear un DF a parti de un RDD no tenemos más que pasarle a la función .toDF los nombres de las columnas como lista
df = rdd.toDF(['numero', 'cuadrado'])

# Podemos ver el esquema de los datos, que nos informa del tipo de datos y de si acepta nulos o no
df.printSchema()

# Podemos ver el DF
df.show()

root
 |-- numero: long (nullable = true)
 |-- cuadrado: long (nullable = true)

+------+--------+
|numero|cuadrado|
+------+--------+
|     0|       0|
|     1|       1|
|     2|       4|
|     3|       9|
|     4|      16|
|     5|      25|
|     6|      36|
|     7|      49|
|     8|      64|
|     9|      81|
+------+--------+



##### También podemos crear un DF a partir de un RDD definiendo nosotros el esquema

In [7]:
rdd1 = sc.parallelize([(1, 'Jose', 35.5), (2, 'Teresa', 54.3), (3, 'Katia', 12.7)])

In [8]:
# Método 1: creamos el esquema a partir de las classes de pyspark.sql.types

esquema1 = StructType(
    [
     StructField('id', IntegerType(), True),
     StructField('nombre', StringType(), True),
     StructField('saldo', DoubleType(), True)
    ]
)

In [9]:
# Método 2: creamos el esquema a partir de un string

esquema2 = "`id` INT, `nombre` STRING, `saldo` DOUBLE"

In [10]:
# Finalmente, para crear un DF aplicando nuestro esquema, usamos la función .createDtaFrame, pasándole el RDD y el esquema

df1 = spark.createDataFrame(rdd1, schema=esquema1)

df1.printSchema()

df1.show()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



In [11]:
df1 = spark.createDataFrame(rdd1, schema=esquema2)

df1.printSchema()

df1.show()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



### Crear un DF a partir de fuentes de datos

Las dos clases principales en SparkSQL para leer y escribir datos son DataFrameReader y DataFrameWriter respectivamente.

#### DataFrameReader

* Una instancia de la clase DataFrameReader está disponible como read en la sesión de Spark y la podemos invocar a través de **spark.read**

* El patrón común para interactuar con DataFrameReader es **spark.read.format(...).option('key', 'value').schema(...).load()**

    * **.format(...)** no es opcional. Puede ser una de las fuente de datos integradas o un formato de dato personalizado

        * **Formato integrado:** se puede usar un nombre corto como por ejemplo json, parquet, jdbc, orc, csv, text...
        * **Formato personalizado:** debe proporcionar un nombre completo
<br>.
    * **.option('key', 'value')** es opcional porque DataFrameReader tiene un conjunto de opciones predeterminadas para cada formato de fuente de datos. Podemos anular estos valores predeterminados proporcionando un valor a la función option('key', 'value')

    * **.schema(...)** puede ser opcional porque algunas fuentes de datos tienen el esquema incrustado dentro de los archivos de datos, podemos pensar en .parquet u .orc En estos casos el esquema se infiere automáticamente, para otros casos es posible que deba proporcionar un esquema.

* Para leer los datos hay dos alternativas aplicables a todos los formatos:

    * **spark.read.<extension>("<path>")** por ejemplo, spark.read.json("/path/to/file.json")
    * **spark.read.format("<extension>").load("<path>")** por ejemplo spark.read.format("json").load("/path/to/file.json")

